In [6]:
import os
import json
from pathlib import Path
from IPython.display import display, Image, clear_output
import ipywidgets as widgets
from pprint import pprint
base_dir = "./results"
model_name = "gpt-5-codex"
# model_name = "gemini-3-pro-preview"
mode = "text"
task = "repair"
dataset = "data/data_demo_renderbench_10"
# 获取所有webname文件夹
result_path = Path(base_dir) / model_name / mode / task
webnames = sorted([d.name for d in result_path.iterdir() if d.is_dir()])

In [ ]:
# 当前索引
current_index = 0

# 创建UI组件
prev_button = widgets.Button(description="上一个")
next_button = widgets.Button(description="下一个")
output = widgets.Output()

# 显示当前webname的数据
def show_webname(index):
    with output:
        clear_output(wait=True)
        
        if not webnames:
            print("没有找到webname文件夹")
            return
        
        webname = webnames[index]
        print(f"当前: {webname}")
        print(f"序号: {index + 1} / {len(webnames)}")
        print("-" * 50)
        
        # 读取result中的json文件
        result_dir = result_path / webname
        json_files = list(result_dir.glob("*.json"))
        
        if not json_files:
            print("未找到结果JSON文件")
            return
        
        with open(json_files[0], 'r', encoding='utf-8') as f:
            result_data = json.load(f)
        print("Task info:")
        print(f"Task Type: {result_data.get('task_type', 'N/A')}")
        pprint(result_data.get('description', 'N/A'), width=80)
        if task in ["edit", "repair"]:
            print("Generate Successfully:" , result_data["generate_successfully"])
        # 读取数据集中的info.json
        info_json_path = Path(dataset) / task / webname / "info.json"
        if info_json_path.exists():
            with open(info_json_path, 'r', encoding='utf-8') as f:
                info_data = json.load(f)
        else:
            print(f"警告: 未找到 {info_json_path}")
            info_data = {}
        
        # 显示模型生成的截图
        print("\n【模型生成结果】")
        if "ans_screenshot" in result_data:
            for screenshot in result_data["ans_screenshot"]:
                ans_path = Path(base_dir) / model_name / mode / task / webname / "ans" / screenshot
                if ans_path.exists():
                    display(Image(filename=str(ans_path)))
                else:
                    print(f"未找到截图: {ans_path}")
        
        # 根据任务类型显示数据集截图
        print("-" * 150)
        if task == "generation":
            print("\n【目标截图 (dst)】")
            if "dst_screenshot" in info_data:
                for screenshot in info_data["dst_screenshot"]:
                    dst_path = Path(dataset) / task / webname / "dst" / screenshot
                    if dst_path.exists():
                        display(Image(filename=str(dst_path)))
                    else:
                        print(f"未找到截图: {dst_path}")
        
        elif task in ["edit", "repair"]:
            print("\n【源截图 (src)】")
            if "src_screenshot" in info_data:
                for screenshot in info_data["src_screenshot"]:
                    src_path = Path(dataset) / task / webname / "src" / screenshot
                    if src_path.exists():
                        display(Image(filename=str(src_path)))
                    else:
                        print(f"未找到截图: {src_path}")
            
            print("\n【目标截图 (dst)】")
            if "dst_screenshot" in info_data:
                for screenshot in info_data["dst_screenshot"]:
                    dst_path = Path(dataset) / task / webname / "dst" / screenshot
                    if dst_path.exists():
                        display(Image(filename=str(dst_path)))
                    else:
                        print(f"未找到截图: {dst_path}")

# 按钮点击事件
def on_prev_clicked(b):
    global current_index
    if current_index > 0:
        current_index -= 1
        show_webname(current_index)

def on_next_clicked(b):
    global current_index
    if current_index < len(webnames) - 1:
        current_index += 1
        show_webname(current_index)

prev_button.on_click(on_prev_clicked)
next_button.on_click(on_next_clicked)

# 显示UI
display(widgets.HBox([prev_button, next_button]))
display(output)

# 显示第一个webname
if webnames:
    show_webname(current_index)


Output()